In [1]:
! rm -rf results/ wandb/ fine_tuned_model_10k/ fine_tuned_model_improved/

In [2]:
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

# กำหนดให้ใช้เฉพาะ GPU 0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

# 1. โหลดและเตรียมข้อมูล
def load_data(business_file, review_file, sample_size=500000):
    # โหลดข้อมูล business
    businesses = {}
    with open(business_file, 'r') as f:
        for line in f:
            business = json.loads(line)
            businesses[business['business_id']] = business

    # โหลดและสุ่มเลือกข้อมูล review
    reviews = []
    with open(review_file, 'r') as f:
        for i, line in enumerate(f):
            if i >= sample_size:
                break
            review = json.loads(line)
            if review['business_id'] in businesses:
                reviews.append(review)

    # สร้าง DataFrame
    df = pd.DataFrame(reviews)
    
    # เพิ่มข้อมูล business เข้าไปใน DataFrame (ยกเว้น business_name)
    df['categories'] = df['business_id'].map(lambda x: businesses[x].get('categories', ''))
    df['city'] = df['business_id'].map(lambda x: businesses[x]['city'])

    # สร้าง full_text ที่ไม่มี business_name
    df['full_text'] = df.apply(lambda row: f"ประเภท: {row['categories']}\nเมือง: {row['city']}\nคะแนน: {row['stars']}/5\nรีวิว: {row['text']}\n\n", axis=1)
    
    return df, businesses

# โหลดและเตรียมข้อมูล
df, businesses = load_data('../yelp_academic_dataset_business.json', '../yelp_academic_dataset_review.json', sample_size=500000)
print(f"จำนวนรีวิวที่โหลด: {len(df)}")

# 2. เตรียมข้อมูลสำหรับ fine-tuning
df['label'] = df['business_id']
label_to_id = {label: id for id, label in enumerate(df['label'].unique())}
id_to_label = {id: label for label, id in label_to_id.items()}
df['label_id'] = df['label'].map(label_to_id)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 3. เตรียม Dataset
train_dataset = Dataset.from_pandas(train_df[['full_text', 'label_id']])
val_dataset = Dataset.from_pandas(val_df[['full_text', 'label_id']])

# 4. เตรียม Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_and_prepare(examples):
    tokenized = tokenizer(examples['full_text'], padding="max_length", truncation=True, max_length=512)
    tokenized['labels'] = examples['label_id']
    return tokenized

tokenized_train = train_dataset.map(tokenize_and_prepare, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize_and_prepare, batched=True, remove_columns=val_dataset.column_names)

# 5. เตรียมโมเดลและ Trainer
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_to_id))
model.to('cuda:0')

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    remove_unused_columns=False,
    no_cuda=False,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)

# 6. Fine-tune โมเดล
print("เริ่มการ fine-tune โมเดล...")
trainer.train()
print("Fine-tune เสร็จสิ้น")

# 7. บันทึกโมเดล
trainer.save_model("./fine_tuned_model_improved")
print("บันทึกโมเดลเรียบร้อย")

/home/ntadmin/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


จำนวนรีวิวที่โหลด: 500000


/home/ntadmin/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|█████████████████████████████████| 100000/100000 [00:26<00:00, 3743.36 examples/s]
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to 

เริ่มการ fine-tune โมเดล...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aekanun2020. Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type dict that is 589912 bytes
wandb: WARNING Serializing object of type dict that is 415152 bytes


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.094600,1.288755,0.842700,0.796936,0.770874,0.842700
2,0.714800,0.759692,0.905730,0.876615,0.859868,0.905730
3,0.510600,0.592889,0.926790,0.904647,0.891990,0.926790
4,0.731200,0.514185,0.936610,0.917385,0.906500,0.936610
5,0.574800,0.485482,0.941400,0.923465,0.913664,0.941400


/home/ntadmin/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ntadmin/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ntadmin/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ntadmin/m

Fine-tune เสร็จสิ้น
บันทึกโมเดลเรียบร้อย


In [3]:
# 8. ฟังก์ชันสำหรับการ matching
def match_business_to_text(user_text, top_n=5):
    inputs = tokenizer(user_text, return_tensors="pt", truncation=True, padding=True, max_length=512).to('cuda:0')
    with torch.no_grad():
        outputs = model(**inputs)
    
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    top_n_probs, top_n_indices = torch.topk(probabilities, top_n)
    
    results = []
    for prob, idx in zip(top_n_probs[0], top_n_indices[0]):
        business_id = id_to_label[idx.item()]
        print ("MODEL's ANSWER ============> ", business_id)
        business_info = businesses[business_id]
        results.append({
            'business_id': business_id,
            'name': business_info['name'],
            'categories': business_info.get('categories', ''),
            'city': business_info['city'],
            'probability': prob.item()
        })
    
    return results

# 9. ฟังก์ชันสำหรับแสดงผลลัพธ์
def display_results(results):
    for i, result in enumerate(results, 1):
        print(f"{i}. {result['name']}")
        print(f"   Business ID: {result['business_id']}")
        print(f"   Categories: {result['categories']}")
        print(f"   City: {result['city']}")
        print(f"   Probability: {result['probability']:.4f}")
        print(f"   Match level: {get_match_level(result['probability'])}")
        
        # แสดงรีวิวทั้งหมดของธุรกิจนี้
        print("\n   Reviews:")
        business_reviews = df[df['business_id'] == result['business_id']]['text'].tolist()
        for j, review in enumerate(business_reviews[:3], 1):  # แสดง 3 รีวิวแรก
            print(f"   Review {j}: {review[:200]}...")  # แสดง 200 ตัวอักษรแรกของแต่ละรีวิว
        
        print("\n" + "="*50 + "\n")

def get_match_level(probability):
    if probability > 0.8:
        return "Very High"
    elif probability > 0.6:
        return "High"
    elif probability > 0.4:
        return "Moderate"
    elif probability > 0.2:
        return "Low"
    else:
        return "Very Low"

In [4]:
# 10. ตัวอย่างการใช้งาน
user_text = "ฉันกำลังมองหาร้านอาหารอิตาเลียนที่มีพาสต้าและพิซซ่าอร่อยๆ"
results = match_business_to_text(user_text)

print(f"User Text: {user_text}\n")
print("Top 5 Matching Businesses:")
display_results(results)

MODEL's ANSWER ============>  mFE9V6LPpsDRUQLEBsBRRA
MODEL's ANSWER ============>  7sKfrJmjG6unAQeWdQUe2w
MODEL's ANSWER ============>  e1jylvnhcMVr1ER8PO4dpA
MODEL's ANSWER ============>  mNu6xYO2bn-rvacN12h9WA
MODEL's ANSWER ============>  CYJRj0lF120-WC3UKsuh5A
User Text: ฉันกำลังมองหาร้านอาหารอิตาเลียนที่มีพาสต้าและพิซซ่าอร่อยๆ

Top 5 Matching Businesses:
1. Pearl of East
   Business ID: mFE9V6LPpsDRUQLEBsBRRA
   Categories: Restaurants
   City: Philadelphia
   Probability: 0.0112
   Match level: Very Low

   Reviews:
   Review 1: I order from this restaraunt all the time usually for their bone in spear ribs, singapore mei fun, chicken wings, house lo mein and order often when it wasnt that good to begin with though it was conv...
   Review 2: Absolutely the best Chinese food around!  Quick delivery too. Love this place!  Nicest people too.  Please reopen!  So sorry to see what accident did to your building!  Hope that no one was hurt!...
   Review 3: This is not the best Chinese f

In [5]:
# 10. ตัวอย่างการใช้งาน
user_text = "I'm looking for an Italian restaurant with delicious pasta and pizza."
results = match_business_to_text(user_text)

print(f"User Text: {user_text}\n")
print("Top 5 Matching Businesses:")
display_results(results)

MODEL's ANSWER ============>  oHAAlU_Yts_avncJK0hJpw
MODEL's ANSWER ============>  vcyyzUqEiCTSrFmHgqvxpg
MODEL's ANSWER ============>  DXYhpEv17mWqSqFjau7vhg
MODEL's ANSWER ============>  aCHS5pnDjChkHpFo4-EfKg
MODEL's ANSWER ============>  OHplb2m_dKPXY46mS0CqJQ
User Text: I'm looking for an Italian restaurant with delicious pasta and pizza.

Top 5 Matching Businesses:
1. Barro's Pizza
   Business ID: oHAAlU_Yts_avncJK0hJpw
   Categories: Italian, Pizza, Restaurants, Chicken Wings, Sandwiches
   City: Marana
   Probability: 0.0100
   Match level: Very Low

   Reviews:
   Review 1: I Love it here. The pizza is fantastic,  and the staff is very friendly. It's a great place to hang out with friends and family....
   Review 2: Barro's is a great place for a couple of slices. The lunch specials are a great value for the money and there is a good variety to chose from....
   Review 3: So far the best tasting pizza place Ive found since I moved to AZ. The flavor, the cheese, the meats, all 